In [184]:
#import needed libraries

import librosa as lb #used for feature extracion and resampling
import numpy as np 
import matplotlib.pyplot as plt
import librosa.display #builds on matplotlib to draw nice spectrograms.
import IPython.display as ipd #used to play .wav files from notebook
import pandas as pd 
# from scipy.io import wavfile #can be used to import .wav files but does not work on 24 bit depth audio files


import soundfile as sf #soundfile enables loading 24 bit depth audio files.
import time #used to measure code runtime. 


The bottleneck in this code is reading the audio file. A single read on my machine takes approximately 0.23 seconds. 
Loading the entire ~5400 wav files takes approximately: 20 minutes


In [189]:
#functions to be used in this exercise:

#function to read wave file and get numpy file:

def ReadAudio(filename):
    x,sr=sf.read(filename)
    x=x.T #transpose to match format of librosa array representation
    #set sample rate at 22050 to reduce memory usage. 
    x=lb.resample(x,sr,22050)
        
    #change any stereo audio to mono.
    if x.shape[0]==2:
        x=lb.to_mono(x)
     
    #change to a list.
    x=list(x)
    return x,22050

#function to find max number of samples over all sound files. Didnt end up using this since it is memory inefficient.
#as Sounds would have to be stored in memory. 
def SampleNumbersMax(Sounds):
    #Sounds in a list of lists.
    maxNow=0
    for sound in Sounds:
        if len(sound)>maxNow:
            maxNow=len(sound)
    return maxNow

#function to check length of sound file (number of samples) and increase it till it is equal to input numSamples
def IncreaseNumSamples(sound,numSamples):
    if len(sound)<numSamples:
        newSound=sound+[0]*(numSamples-len(sound))
    else:
        newSound=sound
    return newSound

#function that takes all sounds and makes them all of equal length: also didnt use this to avoid storing Souds.
def MakeAllEqualLength(Sounds,numSamples):
    NewSounds=[]
    for sound in Sounds:
        NewSounds.append(IncreaseNumSamples(sound,numSamples))
    return NewSounds
    


#function to extract Features:
def getFeatures(Sound):

    C1=lb.feature.melspectrogram(np.array(Sound),sr=sr,hop_length=5000) #see librosa features
    C1=np.reshape(C1,(C1.shape[0]*C1.shape[1],))
    
    return C1

def getFeatures1(Sound):

    C1=lb.feature.chroma_stft(np.array(Sound),sr=sr,hop_length=5000) #see librosa features
    C1=np.reshape(C1,(C1.shape[0]*C1.shape[1],))
    
    return C1


In [190]:


Labels=pd.read_csv('train.csv')

#change to numpy array
Labels=np.array(Labels)


#Build list of IDs and Classes, obeservation with ID ID[i] is in class Class[i]
ID=[]
Class=[]

#unpack labels to IDs and Classes
for val in Labels:
    idnow,classnow=val
    ID.append(idnow)
    Class.append(classnow)

    


        



In [ ]:
#find max sample count of any Sound: no need anymore, as have run and found maxNow (max number of samples till now)=88375
maxNow=0
start=time.time()

for ids in ID:
        
        x,sr=ReadAudio('Train/'+str(ids)+'.wav')
        if len(x)>maxNow:
            maxNow=len(x)
       
endtime=time.time()
print(-start+endtime)

#maxNow turns out to be 88375
#time elapsed for all data=1312 seconds.

In [ ]:
#WeTryThe getFeatures function:

features=[]
for ids in ID:
    x,sr=ReadAudio('Train/'+str(ids)+'.wav') #the bottleneck
    x=IncreaseNumSamples(x,88375)
    features.append(getFeatures(x))

Classes=set(Class)
Classes=list(Classes)




features=np.array(features)
features.shape


trainFeatures=features[:4000,:]
testFeatures=features[4000:,:]

trainLabels=Class[0:4000]
testLabels=Class[4000:]

#change labels from names to 0,1,...,9
for i in range(len(trainLabels)):
    trainLabels[i]=Classes.index(trainLabels[i])
    
for i in range(len(testLabels)):
    testLabels[i]=Classes.index(testLabels[i])    
    
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100,50,30), random_state=1)
clf.fit(trainFeatures, trainLabels)     

PredictedLabels=clf.predict(testFeatures)

print(100*np.mean(PredictedLabels==testLabels))
#percent times predicts correct with using getFeatures to get features

In [180]:
print(100*np.mean(PredictedLabels==testLabels))
#percent times predicted correct while using getFeatures to get features turns out to be 51% using a 100,50,30 MLP.

51.080139372822295


In [171]:
#Use this to hear Audio of File Desired

ipd.Audio('Train/193.wav')

In [192]:
#WeTryThe getFeatures1 function:

features=[]
for ids in ID:
    x,sr=ReadAudio('Train/'+str(ids)+'.wav')
    x=IncreaseNumSamples(x,maxNow)
    features.append(getFeatures1(x))

Classes=set(Class)
Classes=list(Classes)




features=np.array(features)



trainFeatures=features[:4000,:]
testFeatures=features[4000:,:]

trainLabels=Class[0:4000]
testLabels=Class[4000:]

#change labels from names to 0,1,...,9
for i in range(len(trainLabels)):
    trainLabels[i]=Classes.index(trainLabels[i])
    
for i in range(len(testLabels)):
    testLabels[i]=Classes.index(testLabels[i])    
    
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100,50,30), random_state=1)
clf.fit(trainFeatures, trainLabels)     

PredictedLabels=clf.predict(testFeatures)

print(100*np.mean(PredictedLabels==testLabels))
#percent times predicted correct while using getFeatures1 to get features turns out to be 12% using a 100,50,30 MLP.

C:\Users\Marwan\Anaconda3\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


KeyboardInterrupt: 

88375

10